<a href="https://colab.research.google.com/github/Anshuman-37/Twitter_Sentiment_Analysis/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading the data from Kaggle

In [7]:
# !pip install kaggle --upgrade
# !cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d manchunhui/us-election-2020-tweets

 98% 345M/353M [00:03<00:00, 106MB/s] 
100% 353M/353M [00:03<00:00, 119MB/s]


## 